**# Ejemplo de Árbol de decisión**

In [1]:
######################
# adquirir datos
######################

import pandas as pd
datos = pd.read_csv("./data/diabetes.csv") # dataframe

# Voy a categorizar en funcion de los casos sin diabetes (casos negativos)
casos_negativos = datos.sort_values(by='Outcome')
cantidad_casos_negativos = casos_negativos['Outcome'].value_counts(sort=False,ascending=True)[0]
casos_negativos = casos_negativos[:cantidad_casos_negativos]
# print(casos_negativos)

In [2]:
import numpy as np
# Indagacion de los datos
# Analizo los datos en busca de ceros y faltantes
caracteristicas = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI','DiabetesPedigreeFunction']
cuenta = 0
for caract in caracteristicas:
    descriptor = datos[caract].describe()
    cuenta_de_valores = datos[caract].value_counts()
    cantidad_ceros = cuenta_de_valores[0] if 0 in cuenta_de_valores else 0
    cuenta += cantidad_ceros
    print(f'cantidad de ceros en {caract}: {cantidad_ceros}')
print(f'cuenta:{cuenta}')



cantidad de ceros en Glucose: 5
cantidad de ceros en BloodPressure: 35
cantidad de ceros en SkinThickness: 227
cantidad de ceros en Insulin: 374
cantidad de ceros en BMI: 11
cantidad de ceros en DiabetesPedigreeFunction: 0
cuenta:652


Dado que existe una cantidad significativa de ceros, y considerando que 
representan que no se midió, los paso a NaN para que no sean considerados 
en los promedios y algunos calculos

In [3]:
# Asumo que los valores nulos de `caracteristicas` no fueron medidos y los convierto a NaN
for caract in caracteristicas:
    datos[caract] = datos[caract].replace([0, 0.0, '0'], [pd.NA, np.NaN, pd.NA])
    casos_negativos[caract] = casos_negativos[caract].replace([0, 0.0, '0'], np.nan)

# print(datos)

In [4]:
# Pre-procesamiento de datos. Categorizacion de 'Age'
def mapearEdades(edad):
    if (edad >= 0 and edad <= 11): return 0 # 'niño'
    elif (edad >= 12 and edad <= 18): return 1 # 'adolescente'
    elif (edad >= 19 and edad <= 25): return 2 # 'joven'
    elif (edad >= 26 and edad <= 59): return 3 # 'adulto'
    elif (edad >= 60 and edad <= 70): return 4 # 'mayor'
    
datos['Age'] = datos['Age'].apply(mapearEdades)

In [5]:
# Pre-procesamiento de datos. Categorizacion de 'Glucose'
# IMPORTANTE: Categorizo en función de los casos SIN diabetes
descriptor = casos_negativos['Glucose'].describe()

intervalos_glucosa_segun_promedio = [
    descriptor['min'], 
    descriptor['mean'] - descriptor['std'], 
    descriptor['mean'] + descriptor['std'], 
    descriptor['max']
    ]
etiquetas_glucosa = [0, 1, 2]

datos['Glucose'] = pd.cut(datos['Glucose'], bins=intervalos_glucosa_segun_promedio, labels=etiquetas_glucosa)
# print(datos)

In [6]:
# Categorizo el resto de las caracteristicas segun el bloque anterior
caracteristicas_restantes = ['BloodPressure', 'SkinThickness', 'BMI', 'DiabetesPedigreeFunction', 'Insulin']

for caract in caracteristicas_restantes:
    descriptor = casos_negativos[caract].describe()
    # print(descriptor)
    intervalos = [
        descriptor['min'], 
        descriptor['mean'] - descriptor['std'], 
        descriptor['mean'] + descriptor['std'], 
        descriptor['max']
        ]
    etiquetas = [0, 1, 2]

    datos[caract] = pd.cut(datos[caract], bins=intervalos, labels=etiquetas)
    # print(datos)


In [12]:
print(datos)

     index  Pregnancies Glucose BloodPressure SkinThickness Insulin BMI  \
3        3            1       1             1             1       1   1   
4        4            0       2             0             1       1   2   
6        6            3       0             0             1       1   1   
8        8            2       2             1             2       2   1   
14      14            5       2             1             1       1   1   
..     ...          ...     ...           ...           ...     ...  ..   
753    753            0       2             2             2       2   2   
755    755            1       1             2             2       1   1   
760    760            2       1             0             1       0   1   
763    763           10       1             1             2       1   1   
765    765            5       1             1             1       1   1   

    DiabetesPedigreeFunction  Age  Outcome  
3                          1  2.0        0  
4        

In [8]:
# Falta generar categorizacion en funcion de cuartiles

In [9]:
# IMPORTANTE: Falta normalizar

### Splitting de datos

In [10]:
# Splitting de datos

from sklearn.model_selection import train_test_split

caracteristicas_entrada = [
    'Pregnancies', 
    'Glucose',
    'BloodPressure', 
    'SkinThickness', 
    'BMI', 
    'DiabetesPedigreeFunction', 
    'Insulin',
    'Age']
caracteristicas_objetivo = 'Outcome'

X_train, X_test, y_train, y_test = train_test_split(
    datos[caracteristicas_entrada],
    datos[caracteristicas_objetivo],
    test_size = 0.3
    )

In [14]:
from unittest import mock
from sklearn import tree
import sklearn with mock.patch("sklearn.utils.validation._assert_all_finite"): pass

arbol_segun_media = tree.DecisionTreeClassifier(criterion='entropy')
arbol_segun_media.fit(X=X_train, y=y_train)

IndentationError: unexpected indent (678129080.py, line 4)